In [ ]:
import pandas as pd
import os
import spacy
from collections import defaultdict
from datetime import datetime
from os.path import exists
from os import mkdir
from multiprocessing import Pool

In [ ]:
import re
def clean_up(text):
        text = text.replace("–", "-")
        text = text.replace(" ,", ",")
        text = text.replace(" .", ".")
        text = text.replace(" ;", ";")
        text = text.replace("\n", "")
        text = re.sub(r" +", " ", text)
        return text

def normalize_string(text):
    text = re.sub(r"([.!?])", r" \1", text)
    text = re.sub(r"[^a-zA-Z.!?]+", r" ", text)
    text = clean_up(text)
    return text

In [ ]:
#Change the path accordingly
#BART dataset path
# path = 'datasets/final_training_data/bart_score_dataset.csv'
#DAE dataset path
# path = 'datasets/final_training_data/dae_score_dataset.csv'
#Merged dataset path
# path = 'datasets/final_training_data/merged_dataset.csv'
#Baseline dataset path
# path = 'datasets/final_training_data/baseline_dataset.csv'
#BERT dataset path
path = 'datasets/final_training_data/bert_score_dataset.csv'
df = pd.read_csv(path)
df.drop([col for col in df.columns if col.startswith('Unnamed')],axis = 1, inplace = True)
df['source_sentence'] = df['source_sentence'].apply(normalize_string)
df['simplified_sentence'] = df['simplified_sentence'].apply(normalize_string)
df.rename(columns={'source_sentence': 'input_text', 'simplified_sentence': 'target_text'}, inplace=True)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install transformers
!pip install datasets
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import functools
import gc
import logging
import os
import sys
from typing import Dict, Optional, Tuple

import torch
from transformers import (
    DataCollatorForSeq2Seq,
    BertConfig,
    PegasusTokenizer,
    PegasusForConditionalGeneration,
    AdamW
)
from transformers import logging as hf_logging
from datasets import Dataset, load_from_disk, load_metric, DatasetDict
from sklearn.model_selection import train_test_split

In [ ]:
# Load the pre-trained model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

In [ ]:
def preprocess_function(examples):
  inputs = examples['input_text']
  targets = examples['target_text']
  prefix = "simplify: "
  inputs = [prefix + inp for inp in inputs]
  model_inputs = tokenizer(inputs, max_length=120, truncation=True)
  labels = tokenizer(targets, max_length=120, truncation=True)
  labels["input_ids"] = [
      [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
  ]

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
dataset = Dataset.from_pandas(df)
ds_dict = dataset.train_test_split(shuffle=False, test_size=0.2)
final_dataset = ds_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

## Model Training

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=16,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model) 

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset['train'],
    eval_dataset=final_dataset['test'],
    data_collator = data_collator,
)

# Fine-tune the model
trainer.train()
trainer.save_model('models/Pegasus/training/')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.784700,1.609787
200,1.754100,1.593399
300,1.727700,1.577176
400,1.712300,1.561378
